<a href="https://colab.research.google.com/github/anitakumar/ml/blob/main/recomendation_collaborativefiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

header=['user_id','item_id','rating','timestamp']
df=pd.read_csv('u.data',sep='\t',names=header)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items))

Number of users = 943 | Number of movies = 1682


In [3]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

In [4]:
train_data_mat=np.zeros((n_users,n_items))
for line in train_data.itertuples():
    train_data_mat[line[1]-1,line[2]-1]=line[3]
train_data_mat

array([[5., 0., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [5]:
test_data_mat=np.zeros((n_users,n_items))
for line in test_data.itertuples():
    test_data_mat[line[1]-1,line[2]-1]=line[3]
test_data_mat

array([[0., 3., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
def predict(ratings,similarity, type='user'):
  if type=='user':
    mean_user_rating=ratings.mean(axis=1)
    ratings_diff=(ratings-mean_user_rating[:,np.newaxis])
    pred=mean_user_rating[:,np.newaxis]+similarity.dot(ratings_diff)/np.array([np.abs(similarity).sum(axis=1)]).T
  elif type=='item':
    pred=ratings.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])
  return pred


In [9]:
from sklearn.metrics import pairwise_distances
user_similarity=pairwise_distances(train_data_mat)
item_similarity=pairwise_distances(train_data_mat.T)


In [10]:
item_prediction=predict(train_data_mat,item_similarity,type='item')
user_prediction=predict(train_data_mat,user_similarity,type='user')

In [13]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction,ground_truth):
  prediction=prediction[ground_truth.nonzero()].flatten()
  ground_truth=ground_truth[ground_truth.nonzero()].flatten()
  return sqrt(mean_squared_error(prediction,ground_truth))

print('User-based CF RMSE: '+str(rmse(user_prediction,test_data_mat)))
print('Item-based CF RMSE: '+str(rmse(item_prediction,test_data_mat)))

User-based CF RMSE: 3.081779973455565
Item-based CF RMSE: 3.36528137812744
